In [5]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from psql_methods import execute_commands
from Openseas_Methods import *
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import random
from pymongo import MongoClient
from tqdm import tqdm
from datetime import datetime
import requests
from PIL import Image
from io import BytesIO
from IPython.display import display
import os
import pandas as pd
from image_utils import pull_image_from_url
import multiprocessing
skip_list=['ens','base-introduced','fundrop-pass','gemesis','apecoin','dai-stablecoin','uniswap','1inch-token']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
command = ["select * from nfttoimage where slug='boredapeyachtclub'"]
rows  = execute_commands(command)

In [3]:
# Column names for the DataFrame
columns = ['slug', 'token_id', 'url']

# Create a DataFrame from the list of tuples
df = pd.DataFrame(rows, columns=columns)
grouped_data = df.groupby('slug').apply(lambda x: (x['slug'].iloc[0], list(zip(x['token_id'], x['url']))))
args = list(grouped_data)


In [6]:
records = []
num_processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=num_processes) as pool: # Use all cores   
    for result in pool.starmap(pull_image_from_url, args):
        records.append(result)

/global/scratch/tlundy/NFT_Research/nft_venv/lib/python3.8/site-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


# After previous step run feature_extract on newly generated images

In [8]:
#We assume the features have already been computed by the feature_extract_notebook
# model = torch.hub.load('facebookresearch/dino:main', 'dino_vits8')
import feature_extract
import torch
model_string = 'dinov2_vits14'

data_path = '/global/scratch/tlundy/NFT_Research/nft_research/Dino/embedding_test'
out_path = f'/global/scratch/tlundy/NFT_Research/nft_research/Dino/embedding_test_features/{model_string}'
feature_path = out_path+'/testfeat.pth'
features = torch.load(feature_path)
labels = feature_extract.get_labels(data_path)
file_names = feature_extract.get_filenames(data_path)
print(file_names[0])
print(labels)

Dataset ReturnIndexDataset
    Number of datapoints: 10000
    Root location: /global/scratch/tlundy/NFT_Research/nft_research/Dino/embedding_test/val
('boredapeyachtclub', '0')
tensor([0, 0, 0,  ..., 0, 0, 0])


In [34]:
features_list = features.tolist()

# Create a pandas DataFrame
data = {'Label': labels.tolist(), 'Features': features_list,'Collection':[x[0] for x in file_names],
        'NFT_num':[x[1] for x in file_names]}
df = pd.DataFrame(data)

In [64]:
def normalize_vector(vector):
    magnitude = np.linalg.norm(vector)
    if magnitude != 0:
        return vector / magnitude
    else:
        return vector
df['norm_features'] = df['Features'].apply(normalize_vector)

In [ ]:
import numpy as np
from scipy.stats import pearsonr
# Group the DataFrame by 'Label'


# Compute average feature vector for each label
column = 'norm_features'
true_vec = np.mean(df[column].tolist(), axis=0)
true_dists = [np.linalg.norm(true_vec - np.array(row[column])) for _, row in df.iterrows()]
avg_true_dists = sum(true_dists)/len(true_dists)
num_samples = 10
sizes = [5000,2500,500,250,50,25,10,5]
afvs = []
for x in tqdm(range(0,num_samples)):
    samples = []
    samples.append(avg_true_dists)
    for size in sizes:
        sampled_rows = df.sample(size)   
        # Compute average feature vector for the current label
        avg_feature_vector = np.mean(sampled_rows[column].tolist(), axis=0)
        dist = [np.linalg.norm(avg_feature_vector - np.array(row[column])) for _, row in df.iterrows()]
        gaps = [abs(x-y) for x, y in zip(true_dists, dist)]
        avg_dist = sum(gaps)/len(gaps)
        samples.append(avg_dist)
    afvs.append(samples)
df_2 = pd.DataFrame(afvs,columns=['True Dists']+sizes)
# print(average_feature_vectors)

 80%|██████████████████████████████████████████▍          | 8/10 [00:43<00:10,  5.02s/it]

In [63]:
df_2.describe()

,True Dists,5000,2500,500,250,50,25,10,5
count,5.000000e+01,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,4.778930e-01,0.000752,0.001235,0.003197,0.005057,0.010519,0.017335,0.031062,0.048217
std,1.121495e-16,0.000243,0.000312,0.000893,0.001701,0.002785,0.004666,0.008757,0.014084
min,4.778930e-01,0.000432,0.000642,0.001524,0.002545,0.005411,0.010176,0.017307,0.024886
25%,4.778930e-01,0.000604,0.001035,0.002600,0.003966,0.008606,0.013878,0.023917,0.039018
50%,4.778930e-01,0.000671,0.001150,0.003107,0.004611,0.010076,0.016359,0.028424,0.048513
75%,4.778930e-01,0.000856,0.001472,0.003649,0.005870,0.011725,0.021053,0.037386,0.055822
max,4.778930e-01,0.001438,0.002027,0.005901,0.009988,0.019036,0.026887,0.060459,0.098381


In [45]:
df[f'distance_10000'] = [np.linalg.norm(afvs[0][1] - np.array(row['Features'])) for _, row in df.iterrows()]
for size,vec in afvs[1:]:
    df[f'loss_{size}'] = abs(df[f'distance_10000']-[np.linalg.norm(vec - np.array(row['Features'])) for _, row in df.iterrows()])

In [46]:
df.describe()

,Label,distance_10000,loss_5000,loss_2500,loss_500,loss_250,loss_50,loss_25,loss_10,loss_5
count,10000.0,10000.000000,1.000000e+04,1.000000e+04,10000.000000,1.000000e+04,10000.000000,1.000000e+04,10000.000000,10000.000000
mean,0.0,0.477893,9.172778e-04,2.233007e-03,0.003327,3.818623e-03,0.017389,1.689231e-02,0.034215,0.033327
std,0.0,0.071173,6.098213e-04,1.494059e-03,0.002304,2.575069e-03,0.011481,1.261625e-02,0.021775,0.019976
min,0.0,0.324956,2.003225e-07,8.270743e-07,0.000001,6.733999e-07,0.000006,9.724327e-07,0.000049,0.000006
25%,0.0,0.424306,3.933704e-04,1.017917e-03,0.001420,1.670476e-03,0.007795,6.482333e-03,0.016472,0.016797
50%,0.0,0.465317,8.403242e-04,2.015564e-03,0.002951,3.481500e-03,0.015880,1.419889e-02,0.031829,0.032276
75%,0.0,0.521787,1.388247e-03,3.217001e-03,0.004877,5.639650e-03,0.025887,2.523632e-02,0.049248,0.047857
max,0.0,0.786557,2.868575e-03,6.699363e-03,0.011305,1.295937e-02,0.051817,6.047855e-02,0.103424,0.097707
